In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [4]:
!kaggle datasets download crowdflower/twitter-airline-sentiment
!unzip twitter-airline-sentiment.zip


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open twitter-airline-sentiment.zip, twitter-airline-sentiment.zip.zip or twitter-airline-sentiment.zip.ZIP.


In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


In [13]:
df = pd.read_csv('/content/Tweets.csv.zip')
print("Sample data:")
print(df[['text', 'airline_sentiment']].head())


Sample data:
                                                text airline_sentiment
0                @VirginAmerica What @dhepburn said.           neutral
1  @VirginAmerica plus you've added commercials t...          positive
2  @VirginAmerica I didn't today... Must mean I n...           neutral
3  @VirginAmerica it's really aggressive to blast...          negative
4  @VirginAmerica and it's a really big bad thing...          negative


In [14]:
# Step 3: Prepare texts and labels
texts = df['text'].values
labels = df['airline_sentiment'].values


In [15]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
print(f"Label classes: {label_encoder.classes_}")


Label classes: ['negative' 'neutral' 'positive']


In [17]:
# Step 4: Tokenize and pad sequences
vocab_size = 10000
max_length = 50
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=max_length, padding='post')


In [18]:
# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded, labels_encoded, test_size=0.2, random_state=42)


In [19]:
# Step 6: Build LSTM model
embedding_dim = 64

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Step 7: Train model
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


Epoch 1/5
366/366 ━━━━━━━━━━━━━━━━━━━━ 16s 36ms/step - accuracy: 0.6257 - loss: 0.9203 - val_accuracy: 0.7292 - val_loss: 0.6387
Epoch 2/5
366/366 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.7480 - loss: 0.6004 - val_accuracy: 0.7756 - val_loss: 0.5705
Epoch 3/5
366/366 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.8421 - loss: 0.4338 - val_accuracy: 0.7913 - val_loss: 0.5337
Epoch 4/5
366/366 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.8771 - loss: 0.3417 - val_accuracy: 0.7790 - val_loss: 0.6490
Epoch 5/5
366/366 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.9100 - loss: 0.2735 - val_accuracy: 0.7654 - val_loss: 0.6651


In [21]:
# Step 8: Evaluate on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")

92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7681 - loss: 0.6567
Test accuracy: 0.7653688788414001


In [22]:
# Step 9: Predict on test set
pred_probs = model.predict(X_test)
pred_classes = pred_probs.argmax(axis=1)


92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


In [23]:
# Decode labels
pred_labels = label_encoder.inverse_transform(pred_classes)
true_labels = label_encoder.inverse_transform(y_test)


In [24]:
# Step 10: Show some predictions
print("\nSome predictions on test set:")
for i in range(10):
    print(f"Text: {tokenizer.sequences_to_texts([X_test[i]])[0]}")
    print(f"True label: {true_labels[i]} - Predicted label: {pred_labels[i]}")
    print("---")


Some predictions on test set:
Text: southwestair you're my early frontrunner for best airline oscars2016 <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>
True label: positive - Predicted label: positive
---
Text: usairways how is it that my flt to ewr was cancelled flightled yet flts to nyc from usairways are still flying <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>
True label: negative - Predicted label: negative
---
Text: jetblue what is going on with your bdl to dca flights yesterday and today why is every single one getting delayed <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>

In [25]:
# Step 11: Detailed classification report
print("\nClassification report:")
print(classification_report(y_test, pred_classes, target_names=label_encoder.classes_))


Classification report:
              precision    recall  f1-score   support

    negative       0.88      0.84      0.86      1889
     neutral       0.51      0.62      0.56       580
    positive       0.69      0.64      0.66       459

    accuracy                           0.77      2928
   macro avg       0.69      0.70      0.70      2928
weighted avg       0.78      0.77      0.77      2928

